In [ ]:
import os
import logging 
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import math 

from scipy.signal import argrelextrema, argrelmax, argrelmin
from scipy.fft import rfft, rfftfreq, irfft

from matplotlib.pyplot import MultipleLocator

In [ ]:
# class realPolarConverter:
#     def __init__(self, )
def Real2Polar(x):
    return np.abs(x), np.angle(x)

def Polar2Real(radii, angles):
    return radii * np.exp(1j*angles)

## artificial wave

In [ ]:
# test a random pulse waveform 
Duration = 1 
Fs = 250 # Sampling Frequency 
Samples = Fs / Duration 

V_angle = 2 * math.pi / Samples 
angle = np.arange(-math.pi, math.pi, V_angle)

In [ ]:
# ppg pulse with low dicrotic notch valley 
a_low_notch = [1, 0.6]
sigma_low_notch = [0.6, 1.2]
mu_low_notch = [-math.pi / 2, math.pi /6]

In [ ]:
y0 = a_low_notch[0] * np.exp(-(((angle - mu_low_notch[0]) / sigma_low_notch[0]) ** 2) / 2)
y1 = a_low_notch[1] * np.exp(-(((angle - mu_low_notch[1]) / sigma_low_notch[0]) ** 2) / 2)

In [ ]:
ppg_low_notch = y0 + y1 

In [ ]:
plt.plot(ppg_low_notch)
plt.plot(y0)
plt.plot(y1)
plt.show()

In [ ]:
argrelmax(ppg_low_notch)[0]

In [ ]:
argrelmax(ppg_low_notch)[0]/Samples

In [ ]:
# artificial fingertips ppg 
amplitude_low_notch = [1, 0.6]
sigma_low_notch = [0.6, 1]
mu_low_notch = [-3*math.pi / 5, math.pi / 6]

y0 = amplitude_low_notch[0] * np.exp(-(((angle - mu_low_notch[0]) / sigma_low_notch[0]) ** 2) / 2)
y1 = amplitude_low_notch[1] * np.exp(-(((angle - mu_low_notch[1]) / sigma_low_notch[1]) ** 2) / 2)

ppg_fingertip = y0 + y1 

In [ ]:
plt.plot(ppg_fingertip)
plt.title("artificial fingertip ppg")
plt.ylabel("amplitude")
# plt.plot(y0)
# plt.plot(y1)
plt.tight_layout()
plt.savefig('artificial_fingertip_ppg.png', dpi=150)
plt.show()

In [ ]:
argrelmax(ppg_fingertip)[0]/Samples

In [ ]:
# artificial wrist ppg
amplitude_high_notch = [2, 1]
sigma_high_notch = [0.6, 1]
mu_high_notch = [-0.4*math.pi, 0.2*math.pi]

y0 = amplitude_high_notch[0] * np.exp(-(((angle - mu_high_notch[0]) / sigma_high_notch[0]) ** 2) / 2)
y1 = amplitude_high_notch[1] * np.exp(-(((angle - mu_high_notch[1]) / sigma_high_notch[1]) ** 2) / 2)

ppg_wrist = y0 + y1 

In [ ]:
plt.plot(ppg_wrist)
plt.title("artificial wrist ppg")
plt.ylabel("amplitude")
# plt.plot(y0)
# plt.plot(y1)
plt.tight_layout()
plt.savefig('artificial_wrist_ppg.png', dpi=150)
plt.show()

In [ ]:
argrelmax(ppg_wrist)[0]/Samples

## FFT 

In [ ]:
# using FFT in scipy.fftpack to extract the wave frequency of raw signal 
from scipy import fftpack
f_s=Fs

X = rfft(ppg_fingertip)
freqs = rfftfreq(len(ppg_fingertip)) * f_s

fig, ax = plt.subplots()

ax.stem(freqs, np.abs(X))
ax.set_title('PPG fingertip')
ax.set_xlabel('Frequency in Hertz [Hz]')
ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
ax.set_xlim(-1, 20)
# ax.set_ylim(-5, 1e5)
plt.tight_layout()
plt.savefig('artificial_fingertip_fft.png', dpi=150)
plt.show()

In [ ]:
# using FFT in scipy.fftpack to extract the wave frequency of raw signal 
from scipy import fftpack
f_s=Fs

X_wrist = rfft(ppg_wrist)
freqs_wrist = rfftfreq(len(ppg_wrist)) * f_s

fig, ax = plt.subplots()

ax.stem(freqs_wrist, np.abs(X_wrist))
ax.set_title('PPG wrist')
ax.set_xlabel('Frequency in Hertz [Hz]')
ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
ax.set_xlim(-1, 20)
# ax.set_ylim(-5, 1e5)
plt.tight_layout()
plt.savefig('artificial_wrist_fft.png', dpi=150)
plt.show()

In [ ]:
X_wrist[:20] 

In [ ]:
X[:20]

In [ ]:
X / X_wrist

In [ ]:
modulis, phases = Real2Polar(X / X_wrist)

In [ ]:
modulis[:10]

In [ ]:
phases[:10]

In [ ]:
plt.plot(freqs[:20], modulis[:20])
plt.xlim(0, 20)
x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.savefig('transfer_modulus.png', dpi=150)
plt.show()

In [ ]:
plt.plot(freqs[:20], phases[:20])
plt.xlim(0, 20)
x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.savefig('transfer_phase.png', dpi=150)
plt.show()

In [ ]:
# amplitude transfer

transfer_amplitude_dict = {str(int(freqs_wrist[ind])): val for ind, val in enumerate(modulis[:11])}

# phase transfer 

transfer_phase_dict = {str(int(freqs_wrist[ind])): val for ind, val in enumerate(phases[:11])}

In [ ]:
transfer_amplitude_dict, transfer_phase_dict

In [ ]:
transfer_res = []
for i, complex_comp in enumerate(X_wrist[:11]):
    delta_amp = transfer_amplitude_dict[str(i)]
    delta_phase = transfer_phase_dict[str(i)]
    
    moduli, phase = Real2Polar(complex_comp)
    amplified_moduli = moduli * delta_amp
    delay_phase = phase + delta_phase
    
    transfer_comp = Polar2Real(amplified_moduli, delay_phase)
    transfer_res.append(transfer_comp)

In [ ]:
transfer_res = np.array(transfer_res)

In [ ]:
transfer_res

In [ ]:
len(X)

In [ ]:
# transfer_X = np.copy(X)
transfer_X = np.zeros(len(X), dtype=np.complex_)
transfer_X[:11] = transfer_res

# pred_ppg_fingertip = irfft(transfer_res)
pred_ppg_fingertip = irfft(transfer_X)
# pred_ppg_fingertip = irfft(X)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(ppg_fingertip, label='ppg')
ax1.set_title('fingertip ppg signal')

ax2.plot(pred_ppg_fingertip, label='transfer waveform')
ax2.set_title('fingertip ppg from transfer function')
# ax2.legend()
plt.savefig('fingertip_vs_predictedfingertip.png', dpi=150)
plt.show()

In [ ]:
X[:20]

In [ ]:
transfer_X[:20]

In [ ]:
len(pred_ppg_fingertip), len(ppg_fingertip)

In [ ]:
argrelmax(pred_ppg_fingertip)[0]/Samples, argrelmax(ppg_fingertip)[0]/Samples